In [3]:
import torch
import dlc_practical_prologue as prologue

In [36]:
#generate train and test data
#target classes: a>b->0   a<=b->1

[train_input,train_target,train_classes,
 test_input,test_target,test_classes]=prologue.generate_pair_sets(1000)
